In [ ]:
import os
import ee
import geemap
Map = geemap.Map()
Map

# Adding and cropping features

In [ ]:
#Load eLTER sites
sites = ee.FeatureCollection('projects/ee-digdgeografo/assets/elter_lyon')

# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')
imageCrop = image.clip(sites)

# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine DEM to map
Map.addLayer(imageCrop, vis_params, 'SRTM DEM')

Map.addLayer(sites, {}, 'eLTER Sites')

In [ ]:
Map.draw_features
#Map.draw_last_feature

In [ ]:
roi = ee.FeatureCollection(Map.draw_features)
roi.geometry()

FrenchImage = image.clip(roi.geometry())
Map.addLayer(FrenchImage, vis_params, 'Cropped image')

# Exporting results

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'SRTMLCaceres.tif')

In [ ]:
geemap.ee_export_image(FrenchImage, filename=filename, scale=30, region=roi.geometry(), file_per_band=True)

In [ ]:
#Make a feature and set some properties.
feature = (
    ee.Feature(ee.Geometry.Point([-6.52599, 37.17605]))
    .set('Observer', 'Diego')
    .set('email', 'digd.geografo@gmail.com')
)

# Overwrite the old properties with a new dictionary.
newDict = {'Observer': 'Diego', 'presence': 1, 'email': 'diegogarcia@ebd.csic.es'}
feature = feature.set(newDict)

# Check the result.
feature.getInfo()

In [ ]:
Map.addLayer(feature)

## Zonal Statistics

In [ ]:
# Load some rasters data
SOSD2017 = ee.Image("projects/ee-digdgeografo/assets/Donanaphpy_2017_SOSD")
MOSD2017 = ee.Image("projects/ee-digdgeografo/assets/Donanaphpy_2017_MAXD")
EOSD2017 = ee.Image("projects/ee-digdgeografo/assets/Donanaphpy_2017_EOSD")

phen2017 = ee.Image.cat([SOSD2017, MOSD2017, EOSD2017])

In [ ]:
roi = ee.FeatureCollection(Map.draw_features)
roi.geometry()

In [ ]:
Map.addLayer(EOSD2017)

In [ ]:
geemap.zonal_stats(
    SOSD2017, roi, 'donana_eosd.csv', statistics_type='MEDIAN', scale=30
)

# Extract values to points

In [ ]:
SOSD2017 = ee.Image("projects/ee-digdgeografo/assets/Donana_2017_SOSD")
SOSD2018 = ee.Image("projects/ee-digdgeografo/assets/Donana_2018_SOSD")
SOSD2019 = ee.Image("projects/ee-digdgeografo/assets/Donana_2019_SOSD")
SOSD2020 = ee.Image("projects/ee-digdgeografo/assets/Donana_2020_SOSD")
SOSD2021 = ee.Image("projects/ee-digdgeografo/assets/Donana_2021_SOSD")

years = [str(year) for year in range(2017, 2022)]

images = ee.List([SOSD2017, SOSD2018, SOSD2019, SOSD2020, SOSD2021])
collection = ee.ImageCollection.fromImages(images)

In [ ]:
#shp = "your/shape/here.shp"
#fc = geemap.shp_to_ee(shp)
#Map.addLayer(fc, {}, 'Your shapefile')

In [ ]:
# Points to shape
out_shp = os.path.join(os.getcwd(), 'points.shp')
geemap.extract_values_to_points(roi, phen2017, out_shp)

In [ ]:
# Points to csv
out_csv = os.path.join(os.getcwd(), 'points.csv')
geemap.extract_values_to_points(roi, phen2017, out_csv)

# Map slider

In [ ]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

esri = ee.ImageCollection(
    'projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS'
)

esri_2017 = esri.filterDate('2017-01-01', '2017-12-31').mosaic()
esri_2018 = esri.filterDate('2018-01-01', '2018-12-31').mosaic()
esri_2019 = esri.filterDate('2019-01-01', '2019-12-31').mosaic()
esri_2020 = esri.filterDate('2020-01-01', '2020-12-31').mosaic()
esri_2021 = esri.filterDate('2021-01-01', '2021-12-31').mosaic()

esri_vis = {'min': 1, 'max': 11, 'palette': 'esri_lulc'}
sites = ee.FeatureCollection('projects/ee-digdgeografo/assets/elter_lyon')
donana = sites.filter(ee.Filter.eq("layer", 'Donana'))


images = ee.List([esri_2017, esri_2018, esri_2019, esri_2020, esri_2021])
collection = ee.ImageCollection.fromImages(images)
clipped = collection.map(lambda image: image.clip(donana))

Map.add_legend(title='ESRI Land Cover', builtin_legend='ESRI_LandCover_TS')
Map

Map = geemap.Map()
Map.ts_inspector(clipped, years, esri_vis, width='80px')
Map.add_legend(title='ESRI Land Cover', builtin_legend='ESRI_LandCover_TS')
Map

# Linked maps

In [ ]:
SOSD2017 = ee.Image("projects/ee-digdgeografo/assets/Donana_2017_SOSD")
MOSD2017 = ee.Image("projects/ee-digdgeografo/assets/Donana_2017_MAXD")
EOSD2017 = ee.Image("projects/ee-digdgeografo/assets/Donana_2017_EOSD")

In [ ]:
palette = ['#54478C', '#2C699A', '#048BA8', '#EFEA5A', '#F1C453', '#F29E4C', '#d00000', 
                '#9d0208', '#B9E769', '#83E377', '#16DB93', '#0DB39E'] 

vis_params = [
    {'min': 17000, 'max': 17365, 'palette': palette},
    {'min': 17000, 'max': 17365, 'palette': palette},
    {'min': 17000, 'max': 17365, 'palette': palette}]

labels = ['SOS Doy 2017', 'MOS Doy 2017', 'EOS Doy 2017']

In [ ]:
geemap.linked_maps(
    rows=1,
    cols=3,
    height="400px",
    center=[37.7726, -6.1578],
    zoom=9,
    ee_objects=[SOSD2017, MOSD2017, EOSD2017],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

# Ndvi2Gif

In [ ]:
import geemap
from ndvi2gif import NdviSeasonality

In [ ]:
Map = geemap.Map()
Map.add_basemap('CartoDB.DarkMatter')
Map

In [ ]:
roi = Map.draw_last_feature
print(roi.geometry().getInfo())

In [ ]:
MyClass = NdviSeasonality(roi, 2016, 2019, 'Sentinel')

In [ ]:
median = MyClass.get_year_composite().median()
vizParams = {'bands': ['autumn', 'spring', 'summer'], 'min': 0.1, 'max': 0.85, 'gamma': [0.95, 1.1, 1]}
Map.addLayer(median, vizParams, 'median')

In [ ]:
wintermax = MyClass.get_year_composite().select('winter').max()
Map.addLayer(wintermax, {'min': 0, 'max': 0.75}, 'winterMax')

In [ ]:
MyClass.get_export()